<a href="https://colab.research.google.com/github/a-stoller/stc510/blob/main/Module_3_Daniels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Module3
# Wrangling of Phoenix Crime Data


In [1]:
import pandas as pd
import pathlib
import numpy as np
import datetime as dt

In [31]:
#load in CSV file containing crime data. Source: https://www.phoenixopendata.com/dataset/crime-data/resource/0ce3411a-2fc6-4302-a33f-167f68608a20?view_id=9dd33103-a106-42f9-8398-390b62a756bb
myCsvPath = pathlib.Path(r'/content/0ce3411a-2fc6-4302-a33f-167f68608a20.csv')
df = pd.read_csv(myCsvPath, engine="python")


# Create new columns to work with the date and time of the incident

In [32]:
# Convert to datetime
df['OCCURRED ON'] = pd.to_datetime(df['OCCURRED ON'], format='%m/%d/%Y %H:%M')

# New column for Year
df["incidentYear"] = df["OCCURRED ON"].dt.year

# New column for day of week 
# 0 is Monday, 6 is Sunday
df["day_of_week"] = df["OCCURRED ON"].dt.weekday

#New column for time of day
conditions = [
    (df['OCCURRED ON'].dt.hour < 5),
    (df["OCCURRED ON"].dt.hour < 12),
    (df["OCCURRED ON"].dt.hour < 17),
    (df["OCCURRED ON"].dt.hour < 20),
    (df["OCCURRED ON"].dt.hour <= 23)]
choices = ['1', '2', '3', '4', '5']
df['timeOfDay'] = np.select(conditions, choices)

#New column for Violent or Non-Violent
conditions = [
  (df['UCR CRIME CATEGORY'] == 'MURDER AND NON-NEGLIGENT MANSLAUGHTER') | (df['UCR CRIME CATEGORY'] =='RAPE') | (df['UCR CRIME CATEGORY'] =='ROBBERY') | (df['UCR CRIME CATEGORY'] =='AGGRAVATED ASSAULT'),
  (df['UCR CRIME CATEGORY'] == 'ARSON') | (df['UCR CRIME CATEGORY'] == 'BURGLARY') | (df['UCR CRIME CATEGORY'] == 'DRUG OFFENSE') | (df['UCR CRIME CATEGORY'] =='LARCENY-THEFT') | (df['UCR CRIME CATEGORY'] =='MOTOR VEHICLE THEFT')
    ]
choices = ['Violent', 'Non-Violent']
df['Violent or Non'] = np.select(conditions, choices)


# Let's calcuate some calculations and build some tables! 

In [4]:
#Filter out 2023, since this year is not complete and is not 'apples to apples' with other years.
dfFiltered = df[df['incidentYear'] != 2023]

#Zip codes as strings, not numbers
dfFiltered['ZIP'] = dfFiltered['ZIP'].astype(str)

# Group by ZIP and incidentYear, and count the number of incidents
dfFiltered.set_index(['ZIP','incidentYear'], inplace=True)
dfGrouped = dfFiltered.groupby(['ZIP', 'incidentYear'])['INC NUMBER'].nunique().reset_index(name='Incidents')

# create a pivot table by Zip code and Year
dfPivot1 = pd.pivot_table(dfGrouped, values='Incidents', index=['ZIP'], columns=['incidentYear']).fillna(0)

# Calculate columns for change in number of incidents since 2016, and % change since 2016
dfPivot1['deltaSince2016'] = dfPivot1[2022] - dfPivot1[2016]
dfPivot1['rateOfChangeSince2016'] = dfPivot1['deltaSince2016'] / dfPivot1[2016]

# Format rate of change as Percent
dfPivot1['rateOfChangeSince2016'] = dfPivot1['rateOfChangeSince2016'].map('{:.2f}%'.format)

#Sort by DeltaSince2016 so the highest increase is at top
dfPivot1.sort_values(by='deltaSince2016', ascending=False, inplace=True)


print("These Zip Codes have the highest change in crime occurrences:",
dfPivot1.head(10)
)

These Zip Codes have the highest change in crime occurrences: incidentYear  2016  2017  2018  2019  2020  2021    2022  deltaSince2016  \
ZIP                                                                        
85008.0        0.0   0.0   0.0   1.0   1.0  19.0  2286.0          2286.0   
85051.0        0.0   0.0   0.0   0.0   1.0  10.0  2021.0          2021.0   
85015.0        0.0   0.0   0.0   1.0   1.0   4.0  2007.0          2007.0   
85041.0        2.0   0.0   1.0   1.0   0.0   9.0  1704.0          1702.0   
85009.0        0.0   0.0   0.0   0.0   3.0  11.0  1644.0          1644.0   
85043.0        0.0   0.0   0.0   3.0   3.0   9.0  1568.0          1568.0   
85017.0        0.0   0.0   0.0   0.0   1.0   8.0  1566.0          1566.0   
85035.0        1.0   0.0   0.0   0.0   0.0   6.0  1545.0          1544.0   
85021.0        0.0   0.0   0.0   1.0   1.0   9.0  1515.0          1515.0   
85022.0        1.0   0.0   0.0   0.0   2.0   3.0  1358.0          1357.0   

incidentYear rateOfChange

<ipython-input-4-d195fd4fc72b>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfFiltered['ZIP'] = dfFiltered['ZIP'].astype(str)


In [18]:
#Pivot table : Crime type by Time of Day
dfPivot2 = pd.pivot_table(df, values='INC NUMBER', columns="timeOfDay",index = 'UCR CRIME CATEGORY',  aggfunc=len)
dfPivot2.columns =['Early Morning (Midnight-5am)', 'Morning(5am-noon)', 'Afternoon(noon-5pm)', 'Evening(5pm-8pm)', 'Night(8pm-midnight)']

In [6]:
#Pivot table : Crime type by Day of Week
dfPivot3 = pd.pivot_table(df, values='INC NUMBER', columns="day_of_week",index = 'UCR CRIME CATEGORY',  aggfunc=len)
dfPivot3.columns = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']


In [22]:
#Pivot table : Crime type by Premise type 
dfPivot4 = pd.pivot_table(df, values='INC NUMBER', columns='UCR CRIME CATEGORY',index = "PREMISE TYPE",  aggfunc=len, fill_value=0)


In [29]:
#Pivot table : Violent or Non violent by Premise Type
dfPivot5 = pd.pivot_table(df, values='INC NUMBER', columns='Violent or Non',index = "PREMISE TYPE",  aggfunc=len, fill_value=0)
#get Percent of vilent crimes for each location
dfPivot5['Percent Violent'] = dfPivot5['Violent'] / (dfPivot5['Violent'] + dfPivot5['Non-Violent'])
# Format rate of change as Percent
dfPivot5['Percent Violent'] = dfPivot5['Percent Violent'].map('{:.2f}%'.format)

# Reporting
Recommended: Convert the pivot table output to interactive tables for easy sorting

# How has crime volume changed over the years for different Zip Codes?

Here are the 10 zip codes with the greatest increase in crime occurrence between 201 and 2022

In [9]:
dfPivot1.head(10)

incidentYear,2016,2017,2018,2019,2020,2021,2022,deltaSince2016,rateOfChangeSince2016
ZIP,,,,,,,,,
85008.0,0.0,0.0,0.0,1.0,1.0,19.0,2286.0,2286.0,inf%
85051.0,0.0,0.0,0.0,0.0,1.0,10.0,2021.0,2021.0,inf%
85015.0,0.0,0.0,0.0,1.0,1.0,4.0,2007.0,2007.0,inf%
85041.0,2.0,0.0,1.0,1.0,0.0,9.0,1704.0,1702.0,851.00%
85009.0,0.0,0.0,0.0,0.0,3.0,11.0,1644.0,1644.0,inf%
85043.0,0.0,0.0,0.0,3.0,3.0,9.0,1568.0,1568.0,inf%
85017.0,0.0,0.0,0.0,0.0,1.0,8.0,1566.0,1566.0,inf%
85035.0,1.0,0.0,0.0,0.0,0.0,6.0,1545.0,1544.0,1544.00%
85021.0,0.0,0.0,0.0,1.0,1.0,9.0,1515.0,1515.0,inf%


# What time of day do different crimes tend to occur?

A strong argument for the addage "nothing good happens after 1 AM"

In [ ]:
dfPivot2

# What day of the week do different crimes occur?

I doubt that Pheonix cops agree with "thank goodness it's Friday!"

In [ ]:
dfPivot3

# Where do different types of crimes tend to occur?

If you are a car that does not want to be stolen, try not to be parked at an apartment!

In [ ]:
dfPivot4

# Where do Violent and NonViolent crimes occur?

Based on proportion and total count of violent crimes, it's probably best to stay off the sidewalks and streets. 

In [30]:
dfPivot5

Violent or Non,Non-Violent,Violent,Percent Violent
PREMISE TYPE,,,
ABANDONED/CONDEMNED STRUCTURE,15,3,0.17%
ADULT ONLY STORE / MOVIE,16,6,0.27%
AIRPORT,282,6,0.02%
ALLEY,220,104,0.32%
AMUSEMENT PARK,11,5,0.31%
...,...,...,...
TRAIN STATION,6,1,0.14%
TRIBAL LANDS,0,1,1.00%
UNKNOWN,234,87,0.27%
